Loading all files and changing there name to "correded","non-correded","damaged" and "non-damaged" according to the the csv file.

In [ ]:
#import data
# import csv
# import os
# path = 'BGS_Images'
# with open('BGS_name.csv') as csvDataFile:
#     csvReader = csv.reader(csvDataFile)
#     i=0
#     for row in csvReader:
#         if i==0:
#             i=2
#             continue
#         filename=row[0]
#         filepath=os.path.join(path,filename)
#         all_files=os.listdir(filepath)
#         for x in all_files:
#             newfilename=x.split(".")[0]+"_"+row[1]+"_"+row[2]+"."+x.split(".")[1]
#             newfilename_path=os.path.join(filepath,newfilename)
#             oldfilename_path=os.path.join(filepath,x)
#             os.rename(oldfilename_path,newfilename_path)

Converting image to gray scale and then making np array of it.

In [31]:
# train images
import csv
import numpy as np
import cv2
import os
import glob



X_train_data = []

# dir_files ="train/"
dir_files = "train/"
dirs = os.listdir(dir_files)
for img in dirs:
        if img.endswith(".JPG"):
            img_array = cv2.imread(os.path.join(dir_files,img), cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(img_array, (28, 28))
    #         img_flatten = resized_image.reshape(-1)
            X_train_data.append(resized_image)
np_X_train_data = np.array(X_train_data)        
        
print(np_X_train_data.shape)

307
(306, 28, 28)


In [32]:
# test images
X_test_data = []
dir_files ="test/"
print(len(os.listdir(dir_files)))

for img in os.listdir(dir_files):
        img_array = cv2.imread(os.path.join(dir_files,img), cv2.IMREAD_GRAYSCALE)
        resized_image = cv2.resize(img_array, (28, 28))
#         img_flatten = resized_image.reshape(-1)
        X_test_data.append(resized_image)
np_X_test_data = np.array(X_test_data)        
        
print(np_X_test_data.shape)

120
(120, 28, 28)


creating labels for train and test images

In [33]:
from PIL import Image
import os, sys
import glob
import numpy as np

Y_train_data = []
dir_files = glob.glob("train/**.JPG")
print(len(dir_files))
for ii in dir_files:
    if ii.find('_Not Corroded_Not Damaged')!=-1:
        Y_train_data.append(2)
    elif ii.find('_Not Corroded_Damaged')!=-1:
        Y_train_data.append(1)
    elif ii.find('_Corroded_Not Damaged')!=-1:
        Y_train_data.append(3)
    elif ii.find('_Corroded_Damaged')!=-1:
        Y_train_data.append(0)

np_Y_train_data = np.array(Y_train_data) 
print(len(Y_train_data))

306
306


In [34]:
Y_test_data = []
dir_files = glob.glob("test/**.JPG")
print(len(dir_files))
for ii in dir_files:
    if ii.find('_Not Corroded_Not Damaged')!=-1:
        Y_test_data.append(2)
    elif ii.find('_Not Corroded_Damaged')!=-1:
        Y_test_data.append(1)
    elif ii.find('_Corroded_Not Damaged')!=-1:
        Y_test_data.append(3)
    elif ii.find('_Corroded_Damaged')!=-1:
        Y_test_data.append(0)

np_Y_test_data = np.array(Y_test_data) 
print(len(Y_test_data))

120
120


In [35]:
N = 50 # how much data of each class
c1, c2, c3, c4= 2, 1, 3, 0

For Training Data

In [36]:
index_of_c1 = np.argwhere(np_Y_train_data == c1).ravel() # index of 2s
index_of_c1 = np.random.choice(index_of_c1, replace=False)

index_of_c2 = np.argwhere(np_Y_train_data == c2).ravel() # index of 3s
index_of_c2 = np.random.choice(index_of_c2, replace=False)

index_of_c3 = np.argwhere(np_Y_train_data == c3).ravel() # index of 4s
index_of_c3 = np.random.choice(index_of_c3, replace=False)

index_of_c4 = np.argwhere(np_Y_train_data == c4).ravel() # index of 4s
index_of_c4 = np.random.choice(index_of_c4, N, replace=False)

x_train = np_X_train_data[np.hstack((index_of_c1, index_of_c2, index_of_c3, index_of_c4))] # subsample
y_train = np_Y_train_data[np.hstack((index_of_c1, index_of_c2, index_of_c3, index_of_c4))] # subsample

print("size of training subset: %s" % str(x_train.shape))

size of training subset: (53, 28, 28)


In [37]:
as_rgb = lambda im: np.tile(np.expand_dims(im,2),(1,1,3))

#example
print(as_rgb(x_train[0,:,:]).shape)

(28, 28, 3)


In [38]:
import skimage
resize = lambda im: skimage.transform.resize(as_rgb(im), (299, 299), preserve_range=False, order=0)

new_x_train = np.concatenate([[resize(x_train[i,:,:])] for i in range(x_train.shape[0])])
print(new_x_train.shape)

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(53, 299, 299, 3)


For Testing Data

In [39]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [40]:
inception = InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(299,299,3)
)

W0718 13:04:21.246817 4627367360 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [41]:
index_of_c1 = np.argwhere(np_Y_test_data == c1).ravel() # index of 2s
index_of_c1 = np.random.choice(index_of_c1, replace=False)

index_of_c2 = np.argwhere(np_Y_test_data == c2).ravel() # index of 3s
index_of_c2 = np.random.choice(index_of_c2, replace=False)

index_of_c3 = np.argwhere(np_Y_test_data == c3).ravel() # index of 4s
index_of_c3 = np.random.choice(index_of_c3, replace=False)

index_of_c4 = np.argwhere(np_Y_test_data == c4).ravel() # index of 4s
index_of_c4 = np.random.choice(index_of_c4, N, replace=False)

x_test = np_X_test_data[np.hstack((index_of_c1, index_of_c2, index_of_c3, index_of_c4))] # subsample
y_test = np_Y_test_data[np.hstack((index_of_c1, index_of_c2, index_of_c3, index_of_c4))] # subsample

print("size of training subset: %s" % str(x_test.shape))

size of training subset: (53, 28, 28)


In [42]:
new_x_test = np.concatenate([[resize(x_test[i,:,:])] for i in range(x_test.shape[0])])
print(new_x_test.shape)

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(53, 299, 299, 3)


In [43]:
training_features = inception.predict(new_x_train)
print(training_features.shape)

(53, 2048)


In [44]:
test_features = inception.predict(new_x_test)
print(test_features.shape)

(53, 2048)


In [45]:
from sklearn.svm import LinearSVC as Classifier

In [46]:
clf = Classifier()

clf.fit(training_features, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [47]:
y_predict = clf.predict(test_features)

In [48]:
acc = clf.score(test_features, y_test)
print("accuracy: %f" % acc)
print("successfully classified: %d/%d" % (np.sum(y_predict == y_test), len(y_test)))

accuracy: 0.943396
successfully classified: 50/53


In [49]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

classifiers = {
    'svm': SVC, 
    'sgd': SGDClassifier, 
    'linearsvm': LinearSVC,
    'decisiontree': DecisionTreeClassifier,
    'naivebayes': GaussianNB
}

In [50]:
for name, Classifer in classifiers.items():
    clf = Classifer()
    clf.fit(training_features, y_train)
    train_acc = clf.score(training_features, y_train)
    test_acc  = clf.score(test_features,  y_test)
    print("classifer: %s\n  training accuracy: %f\n  test accuracy: %f" % (name, train_acc, test_acc))

classifer: svm
  training accuracy: 0.943396
  test accuracy: 0.943396
classifer: sgd
  training accuracy: 1.000000
  test accuracy: 0.943396
classifer: linearsvm
  training accuracy: 1.000000
  test accuracy: 0.943396
classifer: decisiontree
  training accuracy: 1.000000
  test accuracy: 0.867925
classifer: naivebayes
  training accuracy: 1.000000
  test accuracy: 0.943396


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
